<a href="https://colab.research.google.com/github/Vandan1009/python_1/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
pip install pyspark

In [150]:
from pyspark.sql import SparkSession

In [114]:
%%writefile movies.csv
movieId,movieName,year
1,Parmanu,2018
2,URI,2019
3,rrr,2022
4,Bahubali,2015
5,tanhaji,2020

Overwriting movies.csv


In [115]:
%%writefile reviews.csv
movieId,movieInfo,reviews
1,,435
2,,492
3,,444
4,,880
5,,722
6,,888
7,,220
8,,350
9,,619
10,,226

Overwriting reviews.csv


In [116]:
mvd=spark.read.csv("/content/movies.csv",header=True)
rvd=spark.read.csv("/content/reviews.csv",header=True)

In [117]:
mvd.show()

+-------+---------+----+
|movieId|movieName|year|
+-------+---------+----+
|      1|  Parmanu|2018|
|      2|      URI|2019|
|      3|      rrr|2022|
|      4| Bahubali|2015|
|      5|  tanhaji|2020|
+-------+---------+----+



In [118]:
rvd.show()

+-------+---------+-------+
|movieId|movieInfo|reviews|
+-------+---------+-------+
|      1|     NULL|    435|
|      2|     NULL|    492|
|      3|     NULL|    444|
|      4|     NULL|    880|
|      5|     NULL|    722|
|      6|     NULL|    888|
|      7|     NULL|    220|
|      8|     NULL|    350|
|      9|     NULL|    619|
|     10|     NULL|    226|
+-------+---------+-------+



In [129]:
#Clean reviews dataframe on the basis of movies data available. Hint - try left semi join, inner join, right join for joins
data=rvd.join(mvd,"movieId","right")
data.show()

+-------+---------+-------+---------+----+
|movieId|movieInfo|reviews|movieName|year|
+-------+---------+-------+---------+----+
|      1|     NULL|    435|  Parmanu|2018|
|      2|     NULL|    492|      URI|2019|
|      3|     NULL|    444|      rrr|2022|
|      4|     NULL|    880| Bahubali|2015|
|      5|     NULL|    722|  tanhaji|2020|
+-------+---------+-------+---------+----+



In [159]:
from pyspark.sql.functions import col,concat_ws,exp,count

AttributeError: 'NoneType' object has no attribute 'show'

In [130]:
#Fill movieInfo Column by concating movieName & year column (output should be dataframe).  Hint - try expr method
#mvd.withColumn("movieInfo",concat_ws(" ","movieName","year")).show()
data.withColumn("movieInfo",expr("movieName||','||year")).show()


+-------+-------------+-------+---------+----+
|movieId|    movieInfo|reviews|movieName|year|
+-------+-------------+-------+---------+----+
|      1| Parmanu,2018|    435|  Parmanu|2018|
|      2|     URI,2019|    492|      URI|2019|
|      3|     rrr,2022|    444|      rrr|2022|
|      4|Bahubali,2015|    880| Bahubali|2015|
|      5| tanhaji,2020|    722|  tanhaji|2020|
+-------+-------------+-------+---------+----+



In [133]:
 #Find movie with max reviews (output should be string)
 data.groupby("movieName").agg(max("reviews")).first()[0]

'Bahubali'

In [127]:
data.show()

+-------+---------+-------+---------+----+
|movieId|movieInfo|reviews|movieName|year|
+-------+---------+-------+---------+----+
|      1|     NULL|    435|  Parmanu|2018|
|      2|     NULL|    492|      URI|2019|
|      3|     NULL|    444|      rrr|2022|
|      4|     NULL|    880| Bahubali|2015|
|      5|     NULL|    722|  tanhaji|2020|
+-------+---------+-------+---------+----+



In [142]:
#4. Find total reviews (output should be int)
data.select(count("reviews")).collect()[0][0]

5

In [144]:
%%writefile moviesDF.csv
movieId,movieName,year,fansReview,criticsReview
1,Parmanu,2018,200,100
2,URI,2019,300,200
3,rrr,2022,300,200
4,Bahubali,2015,500,400
5,tanhaji,2020,600,500

Writing moviesDF.csv


In [146]:
%%writefile genreDF.csv
movieId,genre
1,tech
2,history
2,action
4,action
3,action
1,science
4,fiction
5,history
5,action
4,mass

Overwriting genreDF.csv


In [153]:
mvID=spark.read.csv("/content/moviesDF.csv",header=True)
ggID=spark.read.csv("/content/genreDF.csv",header=True)

In [154]:
mvID.show()

+-------+---------+----+----------+-------------+
|movieId|movieName|year|fansReview|criticsReview|
+-------+---------+----+----------+-------------+
|      1|  Parmanu|2018|       200|          100|
|      2|      URI|2019|       300|          200|
|      3|      rrr|2022|       300|          200|
|      4| Bahubali|2015|       500|          400|
|      5|  tanhaji|2020|       600|          500|
+-------+---------+----+----------+-------------+



In [155]:
ggID.show()

+-------+-------+
|movieId|  genre|
+-------+-------+
|      1|   tech|
|      2|history|
|      2| action|
|      4| action|
|      3| action|
|      1|science|
|      4|fiction|
|      5|history|
|      5| action|
|      4|   mass|
+-------+-------+



In [172]:
#Find top movies by fansReview to criticsReview ratio. Hint - Add a column for same
sg=mvID.withColumn("ratio",col("fansReview")/col("criticsReview"))
sg.sort("ratio",ascending = False).first()[1]


'Parmanu'

In [179]:
#Order moviesDF by year to get latest movie
mvID.sort("year",ascending=False).first()[1]

'rrr'

In [206]:
#Join dataframes on movieID & find most popular genre
mvID.join(ggID,"movieID").groupBy("genre").count().sort("count",ascending=False).show(1)

+------+-----+
| genre|count|
+------+-----+
|action|    4|
+------+-----+
only showing top 1 row



In [207]:
df2=mvID.join(ggID,"movieID",).groupby("genre").count()
df2.show()

+-------+-----+
|  genre|count|
+-------+-----+
|   mass|    1|
| action|    4|
|fiction|    1|
|history|    2|
|science|    1|
|   tech|    1|
+-------+-----+



In [208]:
print(df2.rdd.first()[0])

mass


In [205]:
df2.rdd.flatMap(lambda x:[x[0]]).collect()

['mass', 'action', 'fiction', 'history', 'science', 'tech']